In [117]:
import pandas as pd
import numpy as np
from scipy import stats
import scipy
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic
from sklearn.preprocessing import StandardScaler
%pylab inline
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations
from statsmodels.sandbox.stats.multicomp import multipletests 

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [36]:
df = pd.read_csv('churn_analysis.csv')
df.head(5)

,Unnamed: 0,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,0,KS,128,415,no,yes,25,265.1,110,45.07,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,1,OH,107,415,no,yes,26,161.6,123,27.47,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,2,NJ,137,415,no,no,0,243.4,114,41.38,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,3,OH,84,408,yes,no,0,299.4,71,50.90,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,4,OK,75,415,yes,no,0,166.7,113,28.34,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      3333 non-null   int64  
 1   state           3333 non-null   object 
 2   account_length  3333 non-null   int64  
 3   area_code       3333 non-null   int64  
 4   intl_plan       3333 non-null   object 
 5   vmail_plan      3333 non-null   object 
 6   vmail_message   3333 non-null   int64  
 7   day_mins        3333 non-null   float64
 8   day_calls       3333 non-null   int64  
 9   day_charge      3333 non-null   float64
 10  eve_mins        3333 non-null   float64
 11  eve_calls       3333 non-null   int64  
 12  eve_charge      3333 non-null   float64
 13  night_mins      3333 non-null   float64
 14  night_calls     3333 non-null   int64  
 15  night_charge    3333 non-null   float64
 16  intl_mins       3333 non-null   float64
 17  intl_calls      3333 non-null   i

In [38]:
df['state']

0       KS
1       OH
2       NJ
3       OH
4       OK
        ..
3328    AZ
3329    WV
3330    RI
3331    CT
3332    TN
Name: state, Length: 3333, dtype: object

In [82]:
df_control = df[df.treatment == 1]
res = []
states_combs = combinations(df_control.state.unique(), 2)

In [83]:
for i in df['state'].unique():
    for j in df['state'].unique():
        if i==j:
            continue
        ct_state_def = pd.crosstab(df_control[(df_control.state == i) | (df_control.state == j)].state, df_control.churn)  
        res.append(stats.chi2_contingency(ct_state_def, correction=False).pvalue)

In [ ]:
for i in states_combs:
    ct_state_def = pd.crosstab(df_control[(df_control.state == i[0]) | (df_control.state == i[1])].state, df_control.churn)  
    res.append(stats.chi2_contingency(ct_state_def, correction=False).pvalue)

In [84]:
(np.array(res) < 0.05).sum()

68

In [85]:
res

[0.07552398753331192,
 0.3418681850357756,
 0.3855103123401462,
 0.3109093724600651,
 0.9279935044417454,
 0.4783036859137494,
 0.024051580332536272,
 0.2554613649566434,
 0.2024306583692768,
 0.4276282765184104,
 0.2821902324706389,
 0.2735146430204451,
 0.37456149180937925,
 0.1792671301924829,
 0.6167690999347494,
 0.20243065836927682,
 0.8830632707719543,
 0.13361440253771584,
 0.6791249326068319,
 0.1546436045787496,
 0.7471514665996195,
 0.17837448691983818,
 0.10018710484992144,
 0.14518224779471892,
 0.17372157772237434,
 0.45194583973362634,
 0.1156180650795173,
 0.044934001242598755,
 0.5597750584844132,
 0.8234512442969397,
 0.07552398753331192,
 0.2613268489836295,
 0.6791249326068319,
 0.0731755545397953,
 0.4276282765184104,
 0.7215797250891101,
 0.1156180650795173,
 0.20243065836927682,
 0.338393599542998,
 0.852914797545359,
 0.03799818419787574,
 0.8120531453899645,
 0.052757652502719445,
 0.3057487934246431,
 0.9632296531731099,
 0.3418681850357756,
 0.173721577722374

In [75]:
list(states_combs)

[]

In [42]:
df[(df.state == i) | (df.state == j)]

,Unnamed: 0,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
377,377,ND,79,408,no,no,0,205.7,123,34.97,...,226.1,106,10.17,6.7,18,1.81,1,2,0.54,False.
464,464,ND,105,415,no,yes,23,193.5,85,32.90,...,272.4,111,12.26,8.5,5,2.30,0,0,0.34,False.
632,632,ND,68,408,no,yes,22,82.5,97,14.03,...,180.0,114,8.10,4.8,4,1.30,3,2,0.42,False.
640,640,ND,88,415,no,no,0,161.5,92,27.46,...,206.2,95,9.28,7.9,4,2.13,2,1,0.60,False.
664,664,ND,147,408,no,no,0,205.3,95,34.90,...,240.6,84,10.83,7.8,4,2.11,1,2,0.62,False.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073,3073,ND,84,408,no,yes,38,193.0,106,32.81,...,260.4,87,11.72,7.4,5,2.00,2,2,0.51,False.
3118,3118,ND,131,408,no,yes,33,177.1,100,30.11,...,253.4,124,11.40,5.2,5,1.40,1,1,0.57,False.
3124,3124,ND,75,408,no,yes,24,225.5,119,38.34,...,270.9,106,12.19,9.4,2,2.54,3,1,0.50,False.
3159,3159,ND,27,415,no,no,0,227.4,67,38.66,...,61.4,109,2.76,7.8,6,2.11,1,0,0.34,False.


In [43]:
df_treatment_1 = df[df.treatment == 1]
states = df_treatment_1.state.unique()
states_crosstab = pd.crosstab(df_treatment_1.state, df_treatment_1.churn)  

In [44]:
def print_states_chi_pvals(correction):
    states_combs = combinations(states, 2)
    states_churn_p = []
    for comb in states_combs:
        subtable = states_crosstab.loc[comb,:]
        states_churn_p.append(stats.chi2_contingency(subtable, correction=correction)[1])
    alpha = 0.05
    states_churn_p = np.array(states_churn_p)
    print('Среднее ',states_churn_p.mean())
    print('У стольких пар значения pvalue меньше', alpha, ':', sum(states_churn_p < alpha))

In [45]:
print_states_chi_pvals(False)

Среднее  0.5018273798739158
У стольких пар значения pvalue меньше 0.05 : 34


In [86]:
print_states_chi_pvals(True)

Среднее  0.7207040857368472
У стольких пар значения pvalue меньше 0.05 : 0


In [88]:
def print_states_fisher_pvals(correction):
    states_combs = combinations(states, 2)
    states_churn_p = []
    for comb in states_combs:
        subtable = states_crosstab.loc[comb,:]
        states_churn_p.append(stats.fisher_exact(subtable)[1])
    alpha = 0.05
    states_churn_p = np.array(states_churn_p)
    print('Среднее ',states_churn_p.mean())
    print('У стольких пар значения pvalue меньше', alpha, ':', sum(states_churn_p < alpha))

In [89]:
print_states_fisher_pvals(1)

Среднее  0.6483383060020681
У стольких пар значения pvalue меньше 0.05 : 10


In [90]:

df[['day_calls','mes_estim']].head()

,day_calls,mes_estim
0,110,0.65
1,123,0.55
2,114,0.72
3,71,0.28
4,113,0.45


In [92]:
stats.pearsonr(df.day_calls, df.mes_estim)

PearsonRResult(statistic=-0.05179435058757263, pvalue=0.0027798836869776986)

In [94]:

stats.spearmanr(df.day_calls, df.mes_estim)

SignificanceResult(statistic=0.043349880533927444, pvalue=0.012317367189170541)

In [95]:
df_treatment_1[['state', 'churn']].head()

,state,churn
0,KS,False.
3,OH,False.
8,LA,False.
12,IA,False.
17,VT,False.


In [96]:

crosstab_state_churn = pd.crosstab(df_treatment_1.state, df_treatment_1.churn)
chi2, p, _, _ = stats.chi2_contingency(crosstab_state_churn)
print('Достигаемый уровень значимости равен', p)

Достигаемый уровень значимости равен 0.7097590042778473


In [106]:
df_treatment_0 = df[df.treatment == 0]
df_treatment_1 = df[df.treatment == 1]
df_treatment_2 = df[df.treatment == 2]

df_treatment_0_churn = (df_treatment_0.churn == 'True.')
df_treatment_1_churn = (df_treatment_1.churn == 'True.')
df_treatment_2_churn = (df_treatment_2.churn == 'True.')


In [127]:
res = []

In [128]:
res.append(stats.mannwhitneyu(df_treatment_0_churn,df_treatment_1_churn).pvalue)

In [129]:
res.append(stats.mannwhitneyu(df_treatment_0_churn,df_treatment_2_churn).pvalue)

In [130]:
res.append(stats.mannwhitneyu(df_treatment_1_churn,df_treatment_2_churn).pvalue)

In [103]:
df_treatment_0.churn.unique()

array(['False.', 'True.'], dtype=object)

In [112]:
(df_treatment_0.churn == 'True.').sum()/(df_treatment_0.churn.size)

0.14563106796116504

In [113]:
(df_treatment_1.churn == 'True.').sum()/(df_treatment_1.churn.size)

0.1640838650865998

In [114]:
(df_treatment_2.churn == 'True.').sum()/(df_treatment_2.churn.size)

0.12511332728921123

In [131]:
reject, p_corrected, a1, a2 = multipletests(res, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [132]:
res

[0.22845579453940912, 0.1565335063039474, 0.00936568781903248]

In [133]:
p_corrected

array([0.31306701, 0.31306701, 0.02809706])

In [134]:
reject

array([False, False,  True])